# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GraphRepresentationGenerator** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from dataset_manager.movie_lens_manager import MovieLensManager
from dataset_manager.kg_manager import ROOT
from llm_manager.vanilla.classifier import VanillaClassifier

In [2]:
EPOCHS = 3
MODEL_NAME = "google/bert_uncased_L-2_H-128_A-2"
BATCH_SIZE_LLM = 256

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [3]:
kg_manager = MovieLensManager()

In [4]:
kg_manager.llm_df[kg_manager.llm_df["split"] == "test"]["labels"].unique()

array([1, 0], dtype=int64)

llm_df = kg_manager.llm_df.merge(kg_manager.target_df[["id", "prompt_feature_title", "prompt_feature_genres"]].rename(columns={"id": "target_id"}), on = "target_id")
llm_df

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-32m.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [5]:
kg_manager.data

HeteroData(
  source={ node_id=[200948] },
  target={
    node_id=[87585],
    x=[87585, 20],
  },
  (source, edge, target)={ edge_index=[2, 32000204] },
  (target, rev_edge, source)={ edge_index=[2, 32000204] }
)

Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [6]:
VANILLA_ROOT = f"{ROOT}/llm/vanilla"

In [7]:
vanilla_bert_classifier = VanillaClassifier(
    df=kg_manager.llm_df,
    source_df=kg_manager.source_df,
    target_df=kg_manager.target_df,
    root_path=VANILLA_ROOT,
    model_name=MODEL_NAME,
)

device cuda
3


Some weights of VanillaBertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of VanillaBertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized because the shapes did not match:
- bert.embeddings.token_type_embeddings.weight: found shape torch.Size([2, 128]) in the checkpoint and torch.Size([3, 128]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
kg_manager.llm_df

,source_id,target_id,split,labels,prompt_feature_title,prompt_feature_genres
0,32460,1336,train,1,Star Trek VI: The Undiscovered Country (1991),"['Action', 'Mystery', 'Sci-Fi']"
1,40961,3603,train,1,"Road Warrior, The (Mad Max 2) (1981)","['Action', 'Adventure', 'Sci-Fi', 'Thriller']"
2,45142,6186,train,1,Holes (2003),"['Adventure', 'Children', 'Comedy', 'Mystery']"
3,13170,19483,train,1,Spring Breakers (2013),"['Comedy', 'Crime', 'Drama']"
4,190580,1879,train,1,"Breakfast Club, The (1985)","['Comedy', 'Drama']"
...,...,...,...,...,...,...
30720189,43147,33025,val,0,Queen of Earth (2015),['Drama']
30720190,62757,31524,val,0,The Last Man on Earth (2011),"['Drama', 'Sci-Fi']"
30720191,140805,66804,val,0,Chekka Chivantha Vaanam (2018),"['Action', 'Crime', 'Thriller']"
30720192,138191,74659,val,0,Wild Kratts: A Creature Christmas (2015),"['Action', 'Adventure', 'Animation', 'Children..."


Next we generate a vanilla llm dataset and tokenize it for training.

In [9]:
dataset_vanilla = kg_manager.generate_vanilla_dataset(
    vanilla_bert_classifier.tokenize_function
)

Loading dataset from disk:   0%|          | 0/69 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/25 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/25 [00:00<?, ?it/s]

In [10]:
dataset_vanilla["train"][0]["prompt"]

"32460[SEP]1336[SEP]Star Trek VI: The Undiscovered Country (1991)[SEP]['Action', 'Mystery', 'Sci-Fi']"

In [11]:
dataset_vanilla

DatasetDict({
    train: Dataset({
        features: ['source_id', 'target_id', 'split', 'labels', 'prompt_feature_title', 'prompt_feature_genres', 'prompt', 'input_ids', 'attention_mask', 'token_type_ranges', 'token_type_ids'],
        num_rows: 17920114
    })
    val: Dataset({
        features: ['source_id', 'target_id', 'split', 'labels', 'prompt_feature_title', 'prompt_feature_genres', 'prompt', 'input_ids', 'attention_mask', 'token_type_ranges', 'token_type_ids'],
        num_rows: 6400040
    })
    test: Dataset({
        features: ['source_id', 'target_id', 'split', 'labels', 'prompt_feature_title', 'prompt_feature_genres', 'prompt', 'input_ids', 'attention_mask', 'token_type_ranges', 'token_type_ids'],
        num_rows: 6400040
    })
})

Next we train the model on the produced dataset. This can be skipped, if already trained ones.

In [12]:
vanilla_bert_classifier.train_model_on_data(
    dataset_vanilla, epochs=EPOCHS, batch_size=BATCH_SIZE_LLM
)

  0%|          | 0/560008 [00:00<?, ?it/s]

c:\Users\MARS\.conda\envs\hauptprojekt\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.697, 'grad_norm': 1.2483288049697876, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}
{'loss': 0.6958, 'grad_norm': 1.1382488012313843, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}
{'loss': 0.6936, 'grad_norm': 1.257224202156067, 'learning_rate': 3e-06, 'epoch': 0.0}
{'loss': 0.6893, 'grad_norm': 1.0931531190872192, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.6857, 'grad_norm': 0.9541659355163574, 'learning_rate': 5e-06, 'epoch': 0.0}
{'loss': 0.6824, 'grad_norm': 0.7910415530204773, 'learning_rate': 6e-06, 'epoch': 0.0}
{'loss': 0.6754, 'grad_norm': 0.9279493689537048, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.6704, 'grad_norm': 0.5385472774505615, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.664, 'grad_norm': 0.37493422627449036, 'learning_rate': 9e-06, 'epoch': 0.0}
{'loss': 0.6603, 'grad_norm': 0.4827374815940857, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 0.6536, 'grad_norm': 0.48204067

  0%|          | 0/25001 [00:00<?, ?it/s]

{'eval_loss': 0.19167956709861755, 'eval_accuracy': 0.9315149592815045, 'eval_runtime': 5671.0543, 'eval_samples_per_second': 1128.545, 'eval_steps_per_second': 4.409, 'epoch': 1.0}
{'loss': 0.1853, 'grad_norm': 0.9226714372634888, 'learning_rate': 4.378829257133053e-05, 'epoch': 1.0}
{'loss': 0.1783, 'grad_norm': 1.0922462940216064, 'learning_rate': 4.3787398929059104e-05, 'epoch': 1.0}
{'loss': 0.1964, 'grad_norm': 0.752463698387146, 'learning_rate': 4.378650528678768e-05, 'epoch': 1.0}
{'loss': 0.1815, 'grad_norm': 0.6892626881599426, 'learning_rate': 4.378561164451625e-05, 'epoch': 1.0}
{'loss': 0.1654, 'grad_norm': 0.7754478454589844, 'learning_rate': 4.378471800224483e-05, 'epoch': 1.0}
{'loss': 0.1675, 'grad_norm': 0.8194307684898376, 'learning_rate': 4.378382435997341e-05, 'epoch': 1.0}
{'loss': 0.1904, 'grad_norm': 0.9842824935913086, 'learning_rate': 4.378293071770198e-05, 'epoch': 1.0}
{'loss': 0.1864, 'grad_norm': 0.8679962754249573, 'learning_rate': 4.378203707543056e-05, 

  0%|          | 0/25001 [00:00<?, ?it/s]

{'eval_loss': 0.18036873638629913, 'eval_accuracy': 0.9361086805707465, 'eval_runtime': 5660.3977, 'eval_samples_per_second': 1130.67, 'eval_steps_per_second': 4.417, 'epoch': 2.0}
{'loss': 0.1658, 'grad_norm': 0.9793777465820312, 'learning_rate': 3.753279667136127e-05, 'epoch': 2.0}
{'loss': 0.1692, 'grad_norm': 0.9367792010307312, 'learning_rate': 3.753190302908984e-05, 'epoch': 2.0}
{'loss': 0.1708, 'grad_norm': 1.0710872411727905, 'learning_rate': 3.753100938681842e-05, 'epoch': 2.0}
{'loss': 0.1658, 'grad_norm': 0.8617454171180725, 'learning_rate': 3.7530115744547e-05, 'epoch': 2.0}
{'loss': 0.1562, 'grad_norm': 0.6207985877990723, 'learning_rate': 3.7529222102275576e-05, 'epoch': 2.0}
{'loss': 0.1658, 'grad_norm': 1.1618945598602295, 'learning_rate': 3.752832846000415e-05, 'epoch': 2.0}
{'loss': 0.1849, 'grad_norm': 0.6978486180305481, 'learning_rate': 3.7527434817732725e-05, 'epoch': 2.0}
{'loss': 0.1622, 'grad_norm': 0.9065653681755066, 'learning_rate': 3.75265411754613e-05, 'e

  0%|          | 0/25001 [00:00<?, ?it/s]

{'eval_loss': 0.17118492722511292, 'eval_accuracy': 0.9387111643052231, 'eval_runtime': 5624.9498, 'eval_samples_per_second': 1137.795, 'eval_steps_per_second': 4.445, 'epoch': 3.0}
{'loss': 0.167, 'grad_norm': 1.0078628063201904, 'learning_rate': 3.127730077139201e-05, 'epoch': 3.0}
{'loss': 0.1733, 'grad_norm': 0.9559326171875, 'learning_rate': 3.127640712912059e-05, 'epoch': 3.0}
{'loss': 0.1559, 'grad_norm': 0.7620539665222168, 'learning_rate': 3.127551348684916e-05, 'epoch': 3.0}
{'loss': 0.173, 'grad_norm': 1.1013115644454956, 'learning_rate': 3.1274619844577737e-05, 'epoch': 3.0}
{'loss': 0.1447, 'grad_norm': 0.7186695337295532, 'learning_rate': 3.1273726202306314e-05, 'epoch': 3.0}
{'loss': 0.1858, 'grad_norm': 0.6982422471046448, 'learning_rate': 3.1272832560034885e-05, 'epoch': 3.0}
{'loss': 0.1691, 'grad_norm': 0.8288446664810181, 'learning_rate': 3.127193891776346e-05, 'epoch': 3.0}
{'loss': 0.1626, 'grad_norm': 1.0978585481643677, 'learning_rate': 3.127104527549204e-05, 'e

KeyboardInterrupt: 

Next we initialize the prompt encoder only classifier. This classifier uses the vanilla prompt and the KGEs for its link prediction.